# เอ็กเซปชัน

ข้อผิดพลาดในโปรแกรมเป็นสิ่งที่เรามักจะป้องกันได้โดยการตรวจสอบสถานะหรือค่าต่าง ๆ ก่อนที่จะดำเนินการต่อ เช่น ตรวจสอบค่าของพารามิเตอร์ที่รับมาว่ามีค่าอยู่ในช่วงที่ถูกต้องหรือไม่ ตรวจสอบค่าผลลัพธ์จากการเรียกเมทอดหรือการทำงานอื่น ๆ ว่าได้ผลลัพธ์ที่ถูกต้องหรือไม่ หรือตรวจสอบค่าของตัวแปรต่าง ๆ ก่อนที่จะดำเนินการว่าค่าเหล่านั้นอยู่ในสถานะที่จะดำเนินการต่อได้หรือไม่

เมื่อเราตรวจสอบแล้วพบว่ามีข้อผิดพลาด เรามีทางเลือกที่จะจัดการกับข้อผิดพลาดนั้นเดี๋ยวนั้น หรือจะรายงานต่อไปให้ส่วนอื่นของโปรแกรมเป็นผู้จัดการ

กรณีที่ข้อผิดพลาดนั้นจัดการได้ทันทีและสามารถดำเนินการต่อไปได้อย่างไม่มีปัญหาก็ควรจะจัดการเดี๋ยวนั้นเลย แต่ถ้าไม่สามารถจัดการได้ การรายงานต่อไปเป็นสิ่งที่ควรกระทำ

การรายงานข้อผิดพลาดต่อไปยังส่วนอื่นของโปรแกรมมีอยู่ 3 แนวทางหลัก ๆ

1. การคืนค่าเป็นค่าผลลัพธ์แสดงความผิดพลาด
2. การกำหนดค่าตัวแปรสถานะบางตัวเพื่อบ่งชี้ว่าเกิดความผิดพลาดขึ้น
3. การใช้กลไกของเอ็กเซปชัน (exception)

แบบที่ 1 และ 2 เป็นการรายงานข้อผิดพลาดแบบที่โค้ดส่วนที่รับข้อผิดพลาดสามารถละเลยได้โดยการไม่ตรวจสอบค่าที่คืนกลับมาหรือตัวแปรสถานะ ซึ่งอาจนำไปสู่ความผิดพลาดต่อเนื่องจากจุดนั้นได้ ทำให้การหาต้นตอของข้อผิดพลาดเมื่อปัญหาแสดงตัวขึ้นมาให้ผู้ใช้เห็นทำได้ยากขึ้น

แบบที่ 3 เป็นกลไกการรายงานข้อผิดพลาดแบบที่บังคับให้โค้ดที่รับข้อผิดพลาดต้องจัดการกับข้อผิดพลาด ไม่สามารถละเลยได้ การละเลยจะทำให้โปรแกรมหยุดทำงานพร้อมรายงานข้อผิดพลาดทันที ไม่ปล่อยให้โปรแกรมทำงานต่อไปในสถานะที่ผิดพลาด ซึ่งรูปแบบนี้เป็นแบบที่เราจะศึกษากันในบทนี้

ตัวอย่างบางส่วนของความผิดพลาดที่นำไปสู่การเกิดเอ็กเซปชันที่เรามักจะพบได้บ่อยได้แก่

- การพยายามใช้งานอ็อบเจกต์ด้วยตัวอ้างอิงที่มีค่าเป็น `null` (`NullPointerException`)
- การพยายามแปลงชนิดอ็อบเจกต์โดยที่ชนิดของอ็อบเจกต์นั้นไม่ได้มีความสัมพันธ์แบบ is-a กับชนิดปลายทางที่เราต้องการแปลง (`ClassCastException`)
- การอ้างอิงนอกขอบเขตสมาชิกของอาร์เรย์ (`ArrayIndexOutOfBoundsException`)
- การส่งอาร์กิวเมนต์ผิดรูปแบบที่เมทอดต้องการ (`IllegalArgumentException`)

การเกิดขึ้นของเอ็กเซปชันอาจจะเกิดจากระบบของ Java เองพบข้อผิดพลาดแล้วสร้างขึ้น หรือไลบรารีที่เราใช้อยู่พบลักษณะผิดปกติแล้วสร้างขึ้น หรือเราสร้างขึ้นเองในโค้ดของเราด้วยคำสั่ง `throw` (โยน)

เมื่อเกิดเอ็กเซปชันขึ้นแล้ว โค้ดของเราต้องจัดการกับเอ็กเซปชันนั้น ถ้าไม่จัดการ เอ็กเซปชันนั้นจะทำให้เมทอดที่กำลังทำงานอยู่จบการทำงานทันทีแล้วส่งเอ็กเซปชันนั้นไปยังเมทอดที่เรียกเมทอดนั้นอีกที กระบวนการนี้จะเกิดต่อกันเป็นทอดถ้าไม่มีโค้ดที่จัดการกับเอ็กเซปชันนั้นดักอยู่เลย จนไปถึงเมทอดสุดท้ายคือ `main()` หรือเมทอดเริ่มต้นในเทรดอื่น ถ้ายังไม่พบโค้ดที่จัดการกับเอ็กเซปชันนั้น โปรแกรมก็จะหยุดการทำงานแล้วแจ้งข้อผิดพลาด โดยแสดงรายการที่เรียกว่า stack trace ซึ่งจะระบุชื่อของเอ็กเซปชันและรายการเมทอดตั้งแต่จุดที่เกิดเอ็กเซปชันไปยังผู้เรียกเมทอดนั้นเป็นทอด ๆ จนมาถึงเมทอดตัวแรกสุด ซึ่งข้อมูลเหล่านี้จะมีประโยชน์ในการติดตามแก้ไขข้อผิดพลาด

โปรแกรมที่มีหลายเทรดจะหยุดทำงานเฉพาะเทรดที่เกิดเอ็กเซปชัน เทรดอื่น ๆ จะยังทำงานต่อไปได้

พิจารณาโค้ดต่อไปนี้

In [ ]:
import java.util.Scanner;

public class DivideByZeroNoExceptionHandling {
    public static int quotient(int numerator, int denominator) {
        return numerator / denominator;
    }
    
    public static void main(String[] args) {
        Scanner input = new Scanner(System.in);
        System.out.print("Enter an integer numerator: ");
        int numerator = input.nextInt();
        System.out.print("Enter an integer denominator: ");
        int denominator = input.nextInt();
        
        int result = quotient(numerator, denominator);
        System.out.printf("%d/%d = %d%n", numerator, denominator, result);
    }
}

เมทอด `quotient()` รับค่าจำนวนเต็ม 2 ค่าแล้วนำมาหารกัน ผลการรันปกติเป็นดังนี้

```
Enter an integer numerator: 100
Enter an integer denominator: 7
100/7 = 14
```

ในกรณีที่เราป้อนข้อมูลค่าหนึ่งเป็นข้อความ จะเกิดเอ็กเซปชันขึ้น และเนื่องจากเราไม่ได้จัดการกับเอ็กเซปชัน โปรแกรมจึงหยุดทำงานและแสดง stack trace ออกมาดังนี้

```
Enter an integer numerator: 100
Enter an integer denominator: seven
Exception in thread "main" java.util.InputMismatchException
	at java.base/java.util.Scanner.throwFor(Scanner.java:860)
	at java.base/java.util.Scanner.next(Scanner.java:1497)
	at java.base/java.util.Scanner.nextInt(Scanner.java:2161)
	at java.base/java.util.Scanner.nextInt(Scanner.java:2115)
	at DivideByZeroNoExceptionHandling.main(DivideByZeroNoExceptionHandling.java:13)
```

เอ็กเซปชันนี้เกิดขึ้นเมื่อเราป้อนค่า `seven` ให้คำสั่ง `input.nextInt()` ซึ่งต้องการข้อมูลที่เป็นตัวเลขจำนวนเต็ม ข้อความจาก stack trace บอกเราว่าเกิดเอ็กเซปชันขึ้นโดยมีชนิดเป็น `InputMismatchException` เกิดขึ้นจากในคลาส `Scanner` ซึ่งเป็นไลบรารีของ Java เอง และเอ็กเซปชันได้ถูกส่งมาจนถึง `main()` ในบรรทัดที่ 13 ซึ่งเป็นการเรียกคำสั่ง `input.nextInt()` และเนื่องจากไม่มีการจัดการกับเอ็กเซปชัน โปรแกรมก็หยุดทำงานในที่สุด

ถ้าเราลองรันใหม่แล้วป้อนค่าตัวหารเป็น `0` ดู

```
Enter an integer numerator: 100
Enter an integer denominator: 0
Exception in thread "main" java.lang.ArithmeticException: / by zero
	at DivideByZeroNoExceptionHandling.quotient(DivideByZeroNoExceptionHandling.java:5)
	at DivideByZeroNoExceptionHandling.main(DivideByZeroNoExceptionHandling.java:15)
```

จะพบว่าเกิด `ArithmeticException` ขึ้น ซึ่งในกรณีนี้เอ็กเซปชันถูกสร้างขึ้นและส่งมาโดยระบบ runtime ของ Java เองจากการที่เราหารจำนวนเต็มด้วย `0`

ในกรณีนี้ stack trace แจ้งว่าเอ็กเซปชันเกิดขึ้นที่บรรทัดที่ 5 ในโปรแกรมของเราในเมทอด `quotient()` และส่งต่อมาที่ `main()` และจบการทำงานลง

## การจัดการกับเอ็กเซปชันเบื้องต้น

เราสามารถจัดการกับเอ็กเซปชันในตัวอย่างนี้ได้โดยใช้คำสั่ง `try` ... `catch` ดังตัวอย่างที่ปรับปรุงใหม่นี้

In [ ]:
import java.util.Scanner;
import java.util.InputMismatchException;

public class DivideByZeroWithExceptionHandling {
    public static int quotient(int numerator, int denominator) {
        return numerator / denominator;
    }
    
    public static void main(String[] args) {
        Scanner input = new Scanner(System.in);
        boolean keepRetrying = true;
        
        do {
            try {
                System.out.print("Enter an integer numerator: ");
                int numerator = input.nextInt();
                System.out.print("Enter an integer denominator: ");
                int denominator = input.nextInt();

                int result = quotient(numerator, denominator);
                System.out.printf("%d/%d = %d%n", numerator, denominator, result);

                keepRetrying = false;
            } catch (InputMismatchException e) {
                System.err.printf("Exception: %s%n", e);
                System.out.println("You must enter an integer. Please try again.");
                input.nextLine(); // discard the input and try again
            } catch (ArithmeticException e) {
                System.err.printf("Exception: %s%n", e);
                System.out.println("Denominator cannot be zero. Please try again.");
            }
        } while (keepRetrying);

    }
}

เราเพิ่มลูป `do` ... `while` เพื่อให้เกิดการป้อนซ้ำจนกว่าป้อนค่าถูกต้องทั้ง 2 ค่า โดยใช้ตัวแปร `keeyRetrying` เป็นตัวกำหนด ถ้าโปรแกรมสามารถทำงานมาได้จนถึงบรรทัดที่กำหนด `keepRetrying = false` ก็จะออกจากลูปโดยไม่ต้องทำซ้ำรอบต่อไป

โค้ดเดิมเราใส่ไว้ในบล็อก `try` และมีโค้ดจัดการเอ็กเซปชันเพิ่มขึ้นมาในบล็อก `catch` โดยทั้งหมดนี้จะอยู่ในลูป `do` ... `while` เพื่อให้เกิดการทำใหม่เมื่อมีข้อผิดพลาด

ผลการรันปกติเป็นดังนี้

```
Enter an integer numerator: 100
Enter an integer denominator: 7    
100/7 = 14
```

ในกรณีที่เราป้อนข้อมูลเป็นข้อความ จะเกิด `InputMismatchException` ขึ้นที่คำสั่ง `input.nextLine()` ในบล็อก `try` การเกิดเอ็กเซปชันในบล็อก `try` จะทำให้โปรแกรมมีการเทียบเอ็กเซปชันนั้นกับบล็อก `catch` ทันที ถ้ามีการประกาศบล็อก `catch` ที่มีชนิดเอ็กเซปชันสอดคล้องกัน โปรแกรมก็จะไปทำงานที่บล็อก `catch` นั้น โดยเอ็กเซปชันที่เกิดขึ้นนั้นจะถูกจับไว้ด้วยตัวแปรที่ประกาศที่บล็อก `catch` ซึ่งในที่นี้ก็คือ `e`

```
Enter an integer numerator: 100
Enter an integer denominator: seven
Exception: java.util.InputMismatchException
You must enter an integer. Please try again.
Enter an integer numerator: 100
Enter an integer denominator: 7
100/7 = 14
```

ในบล็อก `catch` นี้ เราสั่งให้แสดงข้อมูลของเอ็กเซปชันและข้อความแสดงความผิดพลาด และเรียก `input.nextLine()` เพื่อล้างข้อความที่ค้างอยู่ที่อินพุตเนื่องจากเกิดการอ่านที่ผิดพลาด

ลองรันใหม่อีกครั้งแล้วป้อนค่าตัวหารเป็น 0 ได้ผลดังนี้

```
Enter an integer numerator: 100
Enter an integer denominator: 0
Exception: java.lang.ArithmeticException: / by zero
Denominator cannot be zero. Please try again.
Enter an integer numerator: 100
Enter an integer denominator: 7
100/7 = 14
```

เกิด `ArithmeticException` ที่เมทอด `quotient()` ซึ่งถูกเรียกจากในบล็อก `try` ของ `main()` ในกรณีนี้ ตัวเมทอด `quotient()` เองไม่ได้จัดการกับเอ็กเซปชันนั้น เอ็กเซปชันจึงถูกส่งต่อมาที่ผู้ที่เรียก `quotient()` ซึ่งก็คือในส่วนของ `main()` และถูกดักไว้ด้วยบล็อก `catch` ที่ตรงกัน ซึ่งจะแสดงข้อผิดพลาดแล้วกลับไปรับค่าใหม่

สังเกตว่าเมื่อเกิดเอ็กเซปชันขึ้นในบล็อก `try` โค้ดในบล็อกนี้จะไม่ทำงานต่อ ทำให้ไปไม่ถึงคำสั่ง `keepRetrying = false` ค่าของ `keepRetrying` จึงยังเป็น `true` และทำให้ลูปกลับไปวนซ้ำรับค่าใหม่

จากตัวอย่างนี้เราสรุปการจัดการกับเอ็กเซปชันเบื้องต้นได้ดังนี้

1. ใส่โค้ดที่ต้องการให้ทำงานและมีความเสี่ยงที่จะเกิดเอ็กเซปชันขึ้นในบล็อก `try` เมื่อเกิดเอ็กเซปชันขึ้นที่จุดใดในบล็อก `try` โปรแกรมจะไม่ทำงานต่อในโค้ดส่วนที่เหลือในบล็อก
2. กำหนดบล็อก `catch` ที่จะจัดการกับเอ็กเซปชันที่จะเกิดขึ้น โดยการระบุชนิดของเอ็กเซปชันและชื่อตัวแปรที่จะใช้แทนเอ็กเซปชันนั้นถ้าเกิดเอ็กเซปชันขึ้น เราเรียกส่วนประกาศในวงเล็บของ `catch` นั้นว่าพารามิเตอร์ของเอ็กเซปชัน เอ็กเซปชันถือเป็นอ็อบเจกต์ชนิดหนึ่ง เราสามารถจะรับมาเพื่อนำมาจัดการต่อได้
3. ถ้าบล็อก `try` ทำงานสำเร็จตามปกติ หรือถ้าเกิดเอ็กเซปชันขึ้นและทำในบล็อก `catch` เสร็จสิ้นแล้ว โปรแกรมจะทำงานต่อในส่วนที่ต่อจากบล็อก `catch` สุดท้าย

เราสามารถกำหนดบล็อก `catch` กี่บล็อกก็ได้ตามจำนวนชนิดของเอ็กเซปชันที่เราต้องการจัดการ เอ็กเซปชันที่เกิดขึ้นอาจมีชนิดไม่ตรงกับที่ประกาศในบล็อก `catch` แต่ถ้าเอ็กเซปชันที่เกิดขึ้นเป็นซับคลาสของเอ็กเซปชันชนิดที่ประกาศ บล็อก `catch` นั้นก็จะจับเอ็กเซปชันนั้นได้เช่นกัน

เราสามารถประกาศบล็อก `catch` บล็อกเดียวที่จัดการกับเอ็กเซปชันหลายชนิดได้ด้วยรูปแบบนี้

```
catch (Exception1 | Exception2 | Exception3 e) {
    // ...
}
```

โดย _Exception1_, _Exception2_ และ _Exception3_ เป็นชนิดของเอ็กเซปชันที่ต้องการจับ

ในกรณีที่ เอ็กเซปชันที่เกิดขึ้นไม่เข้ากับบล็อก `catch` ใดเลย เอ็กเซปชันนั้นจะถูกส่งต่อไปยังเมทอดที่เรียกเมทอดนี้อีกทอดหนึ่งตามขั้นตอน

> **หมายเหตุ** เอ็กเซปชันทุกชนิดเป็นซับคลาสของ `Throwable` เราสามารถใช้ `catch` กับคลาสชนิดใดก็ได้ที่เป็นซับคลาสของ `Throwable` เราสามารถสร้างคลาสเอ็กเซปชันของเราเองได้เช่นกัน

## ลำดับชั้นของคลาสเอ็กเซปชัน

เอ็กเซปชันทุกชนิดเป็นซับคลาสของคลาส `Exception` ซึ่งเป็นซับคลาสของ `Throwable` อีกที อ็อบเจกต์ที่ใช้ในกลไกจัดการเอ็กเซปชันได้ต้องเป็นอ็อบเจกต์จากคลาสหรือซับคลาสของ `Throwable` เท่านั้น

ซับคลาสโดยตรงของ `Throwable` จะประกอบด้วย `Exception` และ `Error` และมีโครงสร้างของลำดับชั้นดังนี้

![ExceptionHierarchy](ExceptionHierarchy.png)

Java แบ่งลักษณะอาการผิดปกติไว้เป็นแบบร้ายแรงซึ่งอยู่ในกลุ่มคลาส `Error` และแบบที่จัดการได้ซึ่งอยู่ในกลุ่มคลาส `Exception`

กลุ่มคลาส `Error` จะเป็นลักษณะความผิดปกติที่เกิดขึ้นใน JVM และทำให้ไม่สามารถทำงานต่อและกลับเข้าสู่สภาวะปกติได้ เช่น `StackOverflowError` หรือ `OutOfMemoryError` ซึ่งความผิดพลาดลักษณะนี้โดยปกติแล้วจะร้ายแรงเกินกว่าที่โปรแกรมจะจัดการได้

กลุ่มคลาส `Exception` เป็นลักษณะความผิดปกติที่จัดการได้ โดยยังแบ่งได้เป็นเอ็กเซปชันแบบที่ถูกตรวจสอบ (checked exception) กับแบบที่ไม่ถูกตรวจสอบ (unchecked exception) จากแผนผังลำดับชั้นของคลาสเอ็กเซปชันนี้ คลาสที่เป็นอยู่ในกลุ่มของ `RuntimeException` (เป็นซับคลาสของ `RuntimeException`) จะเป็น unchecked exception ส่วนคลาสเอ็กเซปชันที่ไม่ได้อยู่ในกลุ่ม `RuntimeException` จะจัดเป็น checked exception

กลุ่มที่เป็น checked exeption มักจะเป็นข้อผิดพลาดในลักษณะที่อยู่นอกเหนือการควบคุมของโปรแกรม เช่น เปิดไฟล์ไม่ได้ หรือไฟล์ไม่มีอยู่ ซึ่งจะเกิดเป็น `IOException` หรือ `FileNotFoundException` เป็นต้น เอ็กเซปชันกลุ่มนี้ คอมไพเลอร์ Java จะบังคับให้เราเลือกระหว่างการจัดการกับเอ็กเซปชันหรือการประกาศส่งต่อเอ็กเซปชัน ถ้าเราไม่ต้องการจัดการกับเอ็กเซปชันนี้ในเมทอดของเรา เราต้องประกาศว่าเมทอดของเราอาจจะส่งต่อเอ็กเซปชันนี้ได้ ดังตัวอย่างโค้ดนี้

In [ ]:
public Image loadImage(String fileName) throws IOException {
    return ImageIO.read(new File(fileName));
}

การเรียก `ImageIO.read()` อาจทำให้เกิด `IOException` ซึ่งเป็น checked exception ได้ เมทอด `loadImage()` มีทางเลือก 2 ทาง ทางแรกคือจัดการกับเอ็กเซปชันนี้เองโดยใช้ `try` ... `catch` หรือทางที่สอง คือการประกาศว่าเมทอดอาจจะส่งเอ็กเซปชันนี้ออกมาได้ โดยการประกาศ `throws` เอาไว้ที่ส่วนประกาศเมทอด

คอมไพเลอร์ Java จะบังคับให้เราเลือกอย่างใดอย่างหนึ่งในกรณีของ checked exception ซึ่งถ้าเราไม่ทำตามนี้ โปรแกรมก็จะคอมไพล์ไม่ผ่าน

เอ็กเซปชันประเภท checked exception นี้ถ้าเราไม่จัดการเลย จะทำให้เกิดการประกาศ `throws` ต่อกันเป็นทอด เมทอดที่เรียกใช้ `loadImage()` นี้ถ้าไม่จัดการกับเอ็กเซปชันก็ต้องประกาศ `throws` ด้วย และเมทอดที่เรียกเมทอดนี้ต่อก็เช่นกัน

อีกประเภทคือ unchecked exception ซึ่งเป็นกลุ่มที่เป็นซับคลาสของ `RuntimeException` มักจะเป็นความผิดพลาดที่ป้องกันได้ เช่น `ArithmeticException` หรือ `NullPointerException` ซึ่งป้องกันได้โดยการตรวจสอบค่าก่อนการใช้งาน เอ็กเซปชันประเภทนี้คอมไพเลอร์จะไม่บังคับให้เราจัดการหรือประกาศในส่วนประกาศของเมทอด ดังจะเห็นได้จากตัวอย่างเมทอด `quotient()` ในตอนต้นของบทนี้ซึ่งสามารถจะทำให้เกิด `ArithmeticException` จากการหารด้วย `0` ได้ แต่ตัวเมทอดก็ไม่ได้จัดการหรือประกาศส่งต่อแต่อย่างใด

## การโยนเอ็กเซปชัน

นอกเหนือจากเอ็กเซปชันที่เกิดขึ้นจาก JVM และจากการเรียกใช้งานไลบรารีต่าง ๆ แล้ว เรายังสามารถสร้างเอ็กเซปชันขึ้นมาเองได้เช่นกัน ซึ่งเรามักจะใช้กับความผิดพลาดที่เรากำหนดขึ้นเองในโปรแกรมของเรา

เราสามารถสร้างอ็อบเจกต์เอ็กเซปชันแล้วส่งต่อ (เรียกว่าโยนหรือ throw) ได้ด้วยคำสั่ง `throw` ดังนี้

In [ ]:
public Image loadImage(String fileName) throws IOException {
    Image image = ImageIO.read(new File(fileName));
    if (image == null)
        throw new IOException("Unrecognized image type");
    return image;
}

ในตัวอย่างนี้ ถ้าเราพบว่าค่าที่ส่งกลับมาจาก `ImageIO.read()` เป็น `null` เราจะสร้างอ็อบเจกต์เอ็กเซปชันชนิด `IOException` ขึ้นด้วยคำสั่ง `new` พร้อมกับข้อความว่า `"Unrecognized image type"` แล้ว `throw` เอ็กเซปชันนี้ออกไปนอกเมทอดไปยังผู้เรียกใช้เมทอดนี้

## บล็อก finally

ในการจัดการกับเอ็กเซปชันของบล็อก `try` ... `catch` อาจเกิดกรณีได้ 3 แบบ

1. ไม่เกิดเอ็กเซปชันใด ๆ ในบล็อก `try` เลย หลังจบบล็อก `try` โปรแกรมจะทำงานต่อจากส่วน `try` ... `catch` ตามปกติ
2. เกิดเอ็กเซปชันในบล็อก `try` และมีบล็อก `catch` ที่จัดการเอ็กเซปชันนั้น เมื่อทำงานในส่วนบล็อก `catch` นั้นเสร็จสิ้น โปรแกรมจะทำงานต่อจากส่วน `try` ... `catch` ตามปกติ ยกเว้นถ้ามีการ `throw` เอ็กเซปชันอื่นในบล็อก `catch` นั้นอีก
3. เกิดเอ็กเซปชันในบล็อก `try` แต่ไม่มีบล็อก `catch` ที่จัดการเอ็กเซปชันนั้น เมทอดจะหยุดทำงานต่อและส่งเอ็กเซปชันนั้นออกไปยังเมทอดที่เรียก

ในกรณีที่ 3 เราอาจจะยังมีโค้ดบางส่วนที่เราต้องการให้โปรแกรมทำ เช่น อาจจะมีการเปิดการเชื่อมต่อกับเครือข่ายหรือฐานข้อมูลเอาไว้ ซึ่งเราต้องการให้ปิดการเชื่อมต่อไม่ว่าจะเกิดเอ็กเซปชันขึ้นหรือไม่ก็ตาม เพื่อจะได้คืนทรัพยากรนั้นให้กับระบบให้ส่วนอื่นหรือโปรแกรมอื่นนำไปใช้งานได้

ปัญหาลักษณะนี้เราจะแก้ไขด้วยบล็อก `finally` ซึ่งเป็นอีกส่วนหนึ่งของบล็อก `try` ... `catch` ... `finally`

บล็อก `finally` จะเป็นบล็อกที่จะถูกทำงานเสมอไม่ว่าจะเกิดกรณีใดก็ตามใน 3 แบบที่กล่าวมา หรือแม้แต่จะมีการสั่ง `return`, `break` หรือ `continue` ที่จะทำให้เกิดการออกจากบล็อก `try` (ยกเว้นจะเรียก `System.exit()` ให้โปรแกรมจบการทำงานในทันทีเลย)

บล็อก `finally` สามารถจะมีหรือไม่มีก็ได้ แต่บล็อก `try` ที่ไม่มี `catch` เลยจะต้องมี `finally` อยู่ด้วย

เราลองพิจารณาจากตัวอย่างต่อไปนี้

In [ ]:
public class UsingExceptions {
    public static void main(String[] args) {
        try {
            throwException();
        } catch (Exception e) {
            System.err.println("Exception handled in main()");
        }
        
        doesNotThrowException();
    }
    
    public static void throwException() throws Exception {
        try {
            System.out.println("Method throwException()");
            throw new Exception();
        } catch (Exception e) {
            System.err.println("Exception handled in throwException()");
            throw e; // rethrow the caught exception
        } finally {
            System.err.println("Finally executed in throwException()");
        }
    }
    
    public static void doesNotThrowException() {
        try {
            System.out.println("Method doesNotThrowException()");
        } catch (Exception e) {
            System.err.println(e);
        } finally {
            System.err.println("Finally executed in doesNotThrowException()");
        }
        
        System.out.println("End of method doesNotThrowException()");
    }
}

ผลการรันเป็นดังนี้

```
Method throwException()
Exception handled in throwException()
Finally executed in throwException()
Exception handled in main()
Method doesNotThrowException()
Finally executed in doesNotThrowException()
End of method doesNotThrowException()
```

ลำดับการทำงานเป็นดังนี้

1. เมทอด `main()` เรียกเมทอด `throwException()` ในบล็อก `try`
2. เมทอด `throwException()` แสดงข้อความ `"Method throwException()"` และเกิดเอ็กเซปชันขึ้น (สร้างขึ้นเอง) ทำให้เราหลุดจากบล็อก `try` นี้
3. เกิดการ `catch` ขึ้นในเมทอด `throwException()` เอง ซึ่งในบล็อก `catch` จะแสดงข้อความ `"Exception handled in throwException()"` และกลับโยนเอ็กเซปชันนั้นต่ออีกทอด ซึ่งทำให้เราหลุดจากบล็อก `catch` นี้
4. บล็อก `finally` ในเมทอด `throwException()` ทำงานและแสดงข้อความ `"Finally executed in throwException()"` ก่อนจะส่งเอ็กเซปชันที่โยนต่อมาจากบล็อก `catch` กลับไปสู่ `main()`
5. เมทอด `main()` ได้รับเอ็กเซปชันมาจากเมทอด `throwException()`
6. เกิดการ `catch` ขึ้นในเมทอด `main()` โดยแสดงข้อความ `"Exception handled in main()"` แล้วทำงานต่อ
7. เมทอด `main()` เรียกเมทอด `doesNotThrowException()` นอกบล็อก `try` ... `catch`
8. เมทอด `doesNotThrowException()` แสดงข้อความ `"Method doesNotThrowException()"` โดยไม่เกิดเอ็กเซปชันใด ๆ และทำให้ข้ามส่วน `catch` ไป
9. บล็อก `finally` ในเมทอด `doesNotThrowException()` ทำงานและแสดงข้อความ `"Finally executed in doesNotThrowException()"` และจบการทำงานในส่วนของ `try` ... `catch` ... `finally`
10. เมทอด `doesNotThrowException()` แสดงข้อความ `"End of method doesNotThrowException()"`

จะเห็นได้ว่าบล็อก `finally` จะทำงานในทุกกรณี

เราจะใช้บล็อก `finally` ในการจัดการกับทรัพยากรต่าง ๆ ที่เราต้องการคืนสู่ระบบก่อนสิ้นสุดการทำงานของบล็อกหรือเมทอด ซึ่งไม่ว่าการสิ้นสุดการทำงานนั้นจะเป็นในรูปแบบใด (ยกเว้น `System.exit()`) บล็อก `finally` ก็ให้หลักประกันว่าจะต้องทำงานแน่นอน

### การโยนเอ็กเซปชันต่อ

ในโปรแกรมตัวอย่างข้างต้น เราจะเห็นพฤติกรรมหนึ่งของ `catch` ในตัวอย่าง คือการ `throw` เอ็กเซปชันในบล็อก `catch` ซึ่งเราเรียกว่าการส่งต่อหรือโยนต่อ เราอาจจะใช้การทำงานแบบนี้ไม่บ่อยนัก แต่เมื่อใดที่เราต้องการให้การจัดการข้อผิดพลาดเกิดขึ้นในหลายระดับ ทั้งในเมทอดนี้เองหรือในเมทอดที่เรียก เราสามารถใช้การโยนต่อแบบนี้ได้

อีกกรณีหนึ่งที่เราจะใช้การโยนต่อคือการตีความใหม่ เช่น บางครั้งเอ็กเซปชันที่เกิดขึ้นคือ `IOException` แต่เราอาจจะรับเอ็กเซปชันนี้มา แล้วสร้างอ็อบเจกต์จากคลาสเอ็กเซปชันเฉพาะที่เรากำหนดขึ้นใหม่เองเป็น `TargetFileInUseException` แล้วโยนต่อได้ ซึ่งในกรณีนี้อาจจะทำให้ผู้ที่รับเอ็กเซปชันไปต่อสามารถเข้าใจได้ชัดเจนขึ้นว่าเกิดอะไรขึ้น แทนที่จะเป็น `IOException` ที่อาจมีความหมายได้กว้าง ๆ

## รายการย้อนรอยสแต็ก

เมื่อเกิดเอ็กเซปชันขึ้นแล้วไม่มีการ `catch` ภายในขอบเขตของเมทอด เอ็กเซปชันจะถูกส่งย้อนไปยังเมทอดที่เรียกเมทอดนั้นอีกที การส่งย้อนกลับไปนี้ทำให้เกิดการถอยลงของสแต็กที่บันทึกลำดับการเรียกเมทอด การถอยของสแต็กนี้ให้ผลคล้ายกับการสั่ง `return` จากเมทอด ตัวแปรภายในเมทอดจะถูกยกเลิกทั้งหมด และการทำงานจะกลับคืนไปยังผู้เรียก แต่การกลับนี้ถ้าเมทอดที่เรียกไม่จัดการกับเอ็กเซปชัน การย้อนกลับก็จะเกิดขึ้นต่อไปยังเมทอดที่เรียกเมทอดนี้อีกทอด

เราสามารถแสดงข้อมูลการย้อนกลับของเมทอดจาก stack trace ได้โดยไม่ต้องรอให้การย้อนกลับไปถึง `main()` เมื่อเรา `catch` เอ็กเซปชันแล้ว เราสามารถเรียกเมทอด `printStackTrace()` หรือ `getStackTrace()` เพื่อดูข้อมูลการย้อนได้เลย ดังในตัวอย่างนี้

In [ ]:
public class UsingExceptions {
    public static void main(String[] args) {
        try {
            method1();
        } catch (Exception e) {
            System.err.printf("%s%n%n", e.getMessage());
            e.printStackTrace();
            
            // Obtain stack-trace information
            StackTraceElement[] traceElements = e.getStackTrace();

            System.out.printf("%nStack trace from getStackTrace():%n");
            System.out.println("Class\t\tFile\t\t\tLine\tMethod");
            for (StackTraceElement element : traceElements) {
                System.out.printf("%s\t", element.getClassName());
                System.out.printf("%s\t", element.getFileName());
                System.out.printf("%s\t", element.getLineNumber());
                System.out.printf("%s%n", element.getMethodName());
            }
        }
    }
    
    public static void method1() throws Exception {
        method2();
    }
    
    public static void method2() throws Exception {
        method3();
    }
    
    public static void method3() throws Exception {
        throw new Exception("Exception thrown in method3()");
    }
}

ผลการรันตัวอย่างนี้แสดงให้เห็นการย้อนรอยตั้งแต่เมทอด `method3()` ซึ่งเกิดเอ็กเซปชันเป็นที่แรก ไล่ย้อนมา `method2()`, `method1()` และ `main()` ในที่สุด โดยข้อมูลที่แสดงมีทั้งชื่อคลาส ชื่อไฟล์ บรรทัดที่เกิดเอ็กเซปชัน และเมทอดที่เกิดเอ็กเซปชัน

```
Exception thrown in method3()

java.lang.Exception: Exception thrown in method3()
	at UsingExceptions.method3(UsingExceptions.java:32)
	at UsingExceptions.method2(UsingExceptions.java:28)
	at UsingExceptions.method1(UsingExceptions.java:24)
	at UsingExceptions.main(UsingExceptions.java:4)

Stack trace from getStackTrace():
Class           File                    Line    Method
UsingExceptions UsingExceptions.java    32      method3
UsingExceptions UsingExceptions.java    28      method2
UsingExceptions UsingExceptions.java    24      method1
UsingExceptions UsingExceptions.java    4       main
```

## การโยงเอ็กเซปชันเป็นทอด

การส่งเอ็กเซปชันกลับไปที่เมทอดผู้เรียกนั้น นอกจากการจัดการโดยตรงแล้ว เรายังสามารถจัดการแล้วส่งต่ออีกได้ด้วย การส่งต่อทำโดยการ `catch` แล้วสั่ง `throw` ในบล็อก `catch` นั้นอีก ซึ่งเราอาจส่งเอ็กเซปชันเดิมที่รับมาต่อโดยตรง หรือสร้างเอ็กเซปชันใหม่แล้วส่งตัวใหม่ก็ได้

ถ้าเราเลือกสร้างเอกเซปชันใหม่ เราสามารถให้เอ็กเซปชันใหม่เก็บเอ็กเซปชันเก่าเอาไว้ในตัวได้ เพื่อให้มีข้อมูลสำหรับย้อนรอยกลับไปที่ต้นตอของการเกิดเอ็กเซปชันครั้งแรกได้ เราเรียกการสร้างเอ็กเซปชันใหม่โดยให้เก็บข้อมูลของเอ็กเซปชันเดิมไว้ด้วยนี้ว่าการโยงเอ็กเซปชัน (exception chaining)

โค้ดตัวอย่างต่อไปนี้แสดงให้เห็นการโยงเอ็กเซปชัน

In [ ]:
public class UsingChainedExceptions {
    public static void main(String[] args) {
        try {
            method1();
        } catch (Exception e) {
            e.printStackTrace();
        }
    }
    
    public static void method1() throws Exception {
        try {
            method2();
        } catch (Exception e) {
            throw new Exception("Exception thrown in method1()", e);
        }
    }
    
    public static void method2() throws Exception {
        try {
            method3();
        } catch (Exception e) {
            throw new Exception("Exception thrown in method2()", e);
        }
    }
    
    public static void method3() throws Exception {
        throw new Exception("Exception thrown in method3()");
    }
}

เมื่อ `method2()` `catch` เอ็กเซปชันจาก `method3()` แล้ว ในบล็อก `catch` ก็มีการสร้างเอ็กเซปชันใหม่โดยส่งทั้งข้อความและเอ็กเซปชันเดิมเข้าไปเป็นอาร์กิวเมนต์ในการสร้าง `method1()` ก็ทำเช่นเดียวกัน การส่งเอ็กเซปชันเดิมเข้าไปเป็นอาร์กิวเมนต์ทำให้เอ็กเซปชันใหม่เก็บเอ็กเซปชันเดิมไว้เป็นข้อมูลย้อนรอยด้วย ดังจะเห็นได้จากการพิมพ์ stack trace ใน `main()`

```
java.lang.Exception: Exception thrown in method1()
	at UsingChainedExceptions.method1(UsingChainedExceptions.java:14)
	at UsingChainedExceptions.main(UsingChainedExceptions.java:4)
Caused by: java.lang.Exception: Exception thrown in method2()
	at UsingChainedExceptions.method2(UsingChainedExceptions.java:22)
	at UsingChainedExceptions.method1(UsingChainedExceptions.java:12)
	... 1 more
Caused by: java.lang.Exception: Exception thrown in method3()
	at UsingChainedExceptions.method3(UsingChainedExceptions.java:27)
	at UsingChainedExceptions.method2(UsingChainedExceptions.java:20)
	... 2 more
```

## การสร้างคลาสเอ็กเซปชันใหม่

เราสามารถสร้างคลาสเอ็กเซปชันใหม่เพื่อให้สื่อความหมายของความผิดพลาดในโปรแกรมได้ชัดเจนขึ้นได้ คลาสที่สร้างใหม่นี้จะต้อง `extends` จากคลาสเอ็กเซปชันที่มีอยู่แล้ว

คลาสเอ็กเซปชันใหม่ที่สร้างขึ้นมักจะประกอบด้วยคอนสตรักเตอร์ 4 แบบดังนี้

1. แบบที่ไม่รับพารามิเตอร์ และส่งข้อความแสดงความผิดพลาดที่กำหนดแบบโดยปริยายไว้ไปให้กับคอนสตรักเตอร์ของซูเปอร์คลาส
2. แบบที่รับพารามิเตอร์เป็นข้อความแสดงความผิดพลาด และส่งข้อความนั้นไปให้กับคอนสตรักเตอร์ของซูเปอร์คลาส
3. แบบที่รับพารามิเตอร์เป็นข้อความแสดงความผิดพลาดและอ็อบเจกต์ชนิด `Throwable` ซึ่งจะใช้กรณีที่ต้องการโยงเอ็กเซปชันเป็นทอด (exception chaining) แล้วส่งทั้งข้อความและอ็อบเจกต์นี้ต่อไปให้กับคอนสตรักเตอร์ของซูเปอร์คลาส
4. แบบที่รับพารามิเตอร์เป็นอ็อบเจกต์ชนิด `Throwable` ซึ่งจะใช้กรณีที่ต้องการโยงเอ็กเซปชันเป็นทอด แล้วส่งอ็อบเจกต์นี้ต่อไปให้กับคอนสตรักเตอร์ของซูเปอร์คลาส

การสร้างคลาสเอ็กเซปชันใหม่ ควรเลือก `extends` จากคลาสเอ็กเซปชันที่เกี่ยวข้องกับข้อผิดพลาดที่เราต้องการสื่อมากที่สุด เช่น ถ้าเราต้องการสร้างคลาส `DivisionByZeroException` เพื่อแทนกรณีการหารด้วย `0` โดยเฉพาะ เราควรสืบทอดจาก `ArithmeticException` ซึ่งปกติจะเกิดขึ้นเมื่อมีการหารด้วย `0` อยู่แล้ว

กรณีที่ต้องการบังคับให้เกิดการจัดการกับเอ็กเซปชันที่เราสร้างขึ้นเสมอ ควรเลือกที่จะสืบทอดจากกลุ่ม checked exception

## เงื่อนไขก่อนและเงื่อนไขหลัง

การพัฒนาโปรแกรมในความเป็นจริง เวลาที่ใช้ในการพัฒนาจำนวนไม่น้อยจะหมดไปกับการทดสอบและแก้ไขปัญหาในโปรแกรม

มีเครื่องมือต่าง ๆ และหลักการมากมายที่พัฒนาขึ้นมาเพื่ออำนวยความสะดวกในการป้องกัน การค้นหา และการแก้ไขข้อผิดพลาดต่าง ๆ

บางครั้งนักพัฒนาจะกำหนดเงื่อนไขให้กับเมทอดที่เขียนขึ้นว่าสถานะหรือเงื่อนไขขณะที่เรียกเมทอดนั้นควรเป็นอย่างไร และสถานะของค่าต่าง ๆ และผลลัพธ์ของเมทอดนั้นหลังจากเสร็จสิ้นการทำงานของเมทอดเป็นอย่างไร เงื่อนไขของสถานะที่กำหนดขึ้นเหล่านี้เรียกว่าเงื่อนไขก่อน (precondition) และเงื่อนไขหลัง (postcondition)

การกำหนดเงื่อนไขก่อนและเงื่อนไขหลังอาจจะอยู่ในรูปของคอมเมนต์เหนือเมทอด หรืออยู่ในรูปเอกสารประกอบโค้ดโปรแกรมแบบอื่น ๆ

เงื่อนไขก่อนเป็นสิ่งที่เราคาดว่าจะต้องเป็นจริงขณะที่เมทอดถูกเรียก เรารับประกันว่าเมทอดจะทำงานถูกต้องถ้าเงื่อนไขก่อนเป็นจริง แต่ถ้าเงื่อนไขก่อนไม่เป็นจริง ผลการทำงานของเมทอดจะไม่สามารถกำหนดได้เพราะอยู่นอกเหนือขอบเขตการใช้งานปกติตามที่ตั้งใจไว้

เงื่อนไขหลังเป็นสิ่งที่เราคาดว่าจะเป็นผลลัพธ์ ทั้งค่าที่คืนกลับมาและผลที่เกิดขึ้นกับตัวแปรและอ็อบเจกต์ต่าง ๆ ที่เกี่ยวข้อง เงื่อนไขหลังจะระบุว่าค่าเหล่านั้นควรจะเป็นอย่างไร

ถ้าเงื่อนไขก่อนหรือเงื่อนไขหลังไม่เป็นจริง เมทอดมักจะเลือกที่จะโยนเอ็กเซปชัน เพราะถือว่าเกิดเหตุการณ์หรือรูปแบบการใช้งานที่ผิดปกติ

ลองพิจารณาเมทอด `charAt()` ของคลาส `String` ซึ่งรับพารามิเตอร์ชนิด `int` หนึ่งตัวเป็นตำแหน่งบนสตริงที่เราต้องการรู้ค่าตัวอักษร

เงื่อนไขก่อนของ `charAr()` ระบุว่าพารามิเตอร์จะต้องมีค่ามากกว่าหรือเท่ากับ `0` และน้อยกว่าความยาวของสตริงตัวนี้

ถ้าเงื่อนไขก่อนเป็นจริง เงื่อนไขหลังกำหนดว่าเมทอดนี้จะคืนค่าเป็นตัวอักษรในสตริงนี้ ณ ตำแหน่งที่ระบุ

ถ้าเงื่อนไขก่อนไม่เป็นจริง เมทอดจะมีการสร้างเอ็กเซปชันขึ้นเพื่อบอกว่ามีการอ้างอิงตำแหน่งนอกขอบเขตของสตริงตัวนี้

ข้อกำหนดในเงื่อนไขก่อนและเงื่อนไขหลังเหล่านี้เป็นข้อตกลงของผู้พัฒนาว่าพฤติกรรมของเมทอดจะถูกต้องและให้ผลลัพธ์อย่างไรภายใต้เงื่อนไขก่อนที่ถูกต้อง ในความเป็นจริง เมทอดอาจไม่ทำงานเป็นไปตามเงื่อนไขเหล่านี้ได้ถ้ามีข้อผิดพลาดอยู่ภายในเมทอดเอง ดังนั้นเราจึงจำเป็นต้องมีเครื่องมือที่ใช้ทดสอบความถูกต้องของเงื่อนไขก่อนและเงื่อนไขหลังเพื่อจะทดสอบว่าเมทอดทำงานถูกต้องด้วย

## การใช้ assert

ขณะที่เรากำลังทดสอบหรือแก้ไขโปรแกรม เราจะมีข้อสันนิษฐานหรือการคาดการณ์ว่าค่าสถานะต่าง ๆ ของโปรแกรมควรจะมีค่าเป็นอย่างไร ณ แต่ละจุดของโปรแกรม ซึ่งถ้าโปรแกรมทำงานถูกต้อง ค่าต่าง ๆ เหล่านั้นควรจะเป็นไปตามข้อสันนิษฐานหรือการคาดการณ์ของเรา

แต่ในกรณีที่โปรแกรมทำงานไม่ถูกต้อง หรือเรากำลังอยู่ในระหว่างการทดสอบความถูกต้องของโปรแกรม การตั้งข้อสันนิษฐานและการทดสอบข้อสันนิษฐาน ณ จุดต่าง ๆ ในโปรแกรมเป็นสิ่งหนึ่งที่จะช่วยให้การทดสอบของเราสะดวกขึ้น ทำให้เราพบข้อผิดพลาดในการคิดคำนวณได้ง่ายขึ้น

เงื่อนไขก่อนและเงื่อนไขหลังเป็นรูปแบบของข้อสันนิษฐานอย่างหนึ่ง ซึ่งเราสามารถนำมาทดสอบได้ นอกจากนั้นเรายังสามารถทดสอบ ณ จุดใด ๆ ในโปรแกรมก็ได้เช่นกัน

เครื่องมือที่เราใช้ทดสอบนี้คือคำสั่ง `assert` ซึ่ง Java จะมีให้ 2 รูปแบบ

```
assert expression;

assert expression1 : expression2;
```

แบบแรกจะทดสอบเงื่อนไขใน _expression_ และ `throw` `AssertionError` ถ้าเงื่อนไขเป็นเท็จ แบบที่สองจะทดสอบเงื่อนไข _expression1_ และ `throw` `AssertionError` ไปพร้อมข้อความใน _expression2_

ทั้งสองแบบจะทำให้เกิด `AssertionError` ซึ่งเป็นซับคลาสของ `Error` ซึ่งปกติแล้วจะเป็นกลุ่มของคลาสประเภท `Throwable` แบบที่เราจะไม่จัดการ เมื่อเกิดขึ้นแล้วเราจะปล่อยให้โปรแกรมจบการทำงานไปเลย

พิจารณาตัวอย่างต่อไปนี้

In [ ]:
import java.util.Scanner;

public class AssertTest {
    public static void main(String[] args) {
        Scanner input = new Scanner(System.in);
        
        System.out.print("Enter a number between 0 and 10: ");
        int number = input.nextInt();
        
        assert (number >= 0 && number <= 10) : "bad number: " + number;
        
        System.out.printf("You entered %d%n", number);
    }
}

เมื่อเราจะรันโปรแกรมนี้โดยใช้ `assert` เราจะต้องสั่งว่า

```
java -ea AssertionTest
```

ซึ่งจะมีตัวเลือก `-ea` เพิ่มขึ้นมาเป็นตัวบอกว่าให้ใช้ `assert` ด้วย ถ้าเราไม่ระบุตัวเลือกนี้ คำสั่ง `assert` ในโปรแกรมจะไม่ทำงาน ทั้งนี้เพราะ `assert` เป็นเครื่องมือสำหรับทดสอบและแก้ไขข้อผิดพลาดในโปรแกรม ไม่ใช่เครื่องมือสำหรับผู้ใช้โปรแกรมทั่วไป การเปิดใช้ `assert` จะทำให้การประสิทธิภาพการทำงานของโปรแกรมลดลงได้ และอาจส่งผลที่ไม่พึงปรารถนากับผู้ใช้โปรแกรม เช่น โปรแกรมหยุดทำงานขณะกำลังใช้งาน

ผลการรันเป็นดังนี้

```
Enter a number between 0 and 10: 8
You entered 8
```

แต่ถ้าเราใส่ค่าที่ไม่อยู่ในช่วงที่กำหนด จะทำให้เกิด `AssertionError` ขึ้น

```
Enter a number between 0 and 10: 11
Exception in thread "main" java.lang.AssertionError: bad number: 11
	at AssertTest.main(AssertTest.java:10)
```

ตัวอย่างนี้เป็นเพียงการแสดงผลลัพธ์ของ `assert` แต่ไม่ใช่ตัวอย่างแนวทางในการใช้งาน `assert` ในทางปฏิบัติ

เราจะไม่ใช้ `assert` ในการตรวจสอบความผิดพลาดทั่วไป แต่เราจะใช้ในการทดสอบการทำงานของโปรแกรมว่าเป็นไปตามที่เราคาดหรือไม่ โดยการใช้ `assert` ดักไว้ตามจุดต่าง ๆ เพื่อทดสอบว่าค่าสถานะบางอย่างยังเป็นไปตามที่ควรจะเป็น ซึ่งถ้าโปรแกรมทำงานถูกต้อง `assert` ที่อยู่ตามจุดต่าง ๆ เหล่านั้นไม่ควรจะสร้าง `AssertionError` ขึ้นมา แต่ถ้าเกิด `AssertionError` ขึ้นมา เราก็จะเห็นได้ง่ายขึ้นว่าความผิดพลาดเกิดขึ้นที่บริเวณไหน

เราสามารถใช้ `assert` เพื่อระบุเงื่อนไขก่อนและเงื่อนไขหลังได้เช่นกัน และอาจถือเป็นการทำเอกสารประกอบเมทอดไปด้วยในตัว